![](https://static.tacdn.com/img2/brand_refresh/Tripadvisor_lockup_horizontal_secondary_registered.svg)
# Predict TripAdvisor Rating

* Гиль Юлия
* Группа DSPR-28

# Table of Contents

1. [IMPORT](#1)
2. [DATA](#2)
3. [CLEANING AND PREPARING DATA](#3)
    * [Обработка NAN ](#3.1)
    * [Обработка признаков](#3.2)
4. [EDA](#4)
    * [Распределение признаков](#4.1)
    * [Распределение целевой переменной Rating](#4.2)
    * [Корреляция признаков](#4.3)
    * [Анализ номинативных переменных](#4.4)

5. [DATA PREPROCESSING](#5)
6. [MODEL](#6)
7. [SUBMISSION](#7)
8. [SUMMARY](#8)



<a id="1"></a>
# 1. IMPORT

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re

import matplotlib.pyplot as plt
import seaborn as sns 

from itertools import combinations
from scipy.stats import ttest_ind

from sklearn.preprocessing import MultiLabelBinarizer

from math import radians, sin, cos, asin, sqrt # для рассчета расстояний между координатами

%matplotlib inline

import warnings
warnings.simplefilter('ignore')

sns.set()

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

<a id="2"></a>
# 2. DATA

In [ ]:
# Открываем необходимые данные, создаем датафреймы
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'sample_submission.csv')
cities_info = pd.read_csv('/kaggle/input/citiesdata-2/cities_data.csv')

In [ ]:
# Создадим датафрейм data_ta из данных, которые собрали с TripAdvisor
# Открываем каждый файл и добавляем его в data_ta
data_ta = pd.DataFrame()

for dirname, _, filenames in os.walk('/kaggle/input/data-ta/'):
    for filename in filenames:
        temp = pd.read_csv(os.path.join(dirname, filename))
        data_ta = pd.concat([data_ta, temp], ignore_index = True)

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
data_ta.info()

In [ ]:
cities_info.info()

In [ ]:
cities_info.head()

In [ ]:
df_test.head(5)

In [ ]:
data_ta.head()

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# Для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, поэтому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: город 
* Cuisine Style: кухня
* Ranking: ранг ресторана относительно других ресторанов в этом городе
* Price Range: цены в ресторане в 3 категориях
* Number of Reviews: количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

<a id="3"></a>
# 3. CLEANING AND PREPARING DATA

<a id="3.1"></a>
## 3.1. Обработка NAN 

Посмотрим вцелом на наличие пустых значений и определим стратегию работы с ними.

In [ ]:
def intitial_eda_checks(df, missing_percent):
    '''
    Функция принимает на вход датафрейм, а также заданный порог % пустых значений, который хотим обработать. 
    На выход выводит на экран информацию о сумме пустых значений для всех колонок, а также проце
    '''
    if df.isnull().sum().sum() > 0:
        mask_total = df.isnull().sum().sort_values(ascending=False)
        total = mask_total[mask_total > 0]

        mask_percent = df.isnull().mean().sort_values(ascending=False)
        percent = mask_percent[mask_percent > 0]

        series = mask_percent[mask_percent > missing_percent]
        columns = series.index.to_list()

        missing_data = pd.DataFrame(pd.concat(
            [total, round(percent*100, 2)], axis=1, keys=['Количество', '%']))
        print('Сумма и процент значений NaN:\n \n')
        display(missing_data)
    else:
        print('NaN значения не найдены.')

In [ ]:
# Запускаем функцию вывода всех пустых значений
intitial_eda_checks(data, 0)

In [ ]:
# Посмотрим, как распределены пропуски
sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='Blues')

В таблице выведена информация по всем пустым значениям для всех столбцов основного рабочего датасета (data). 

* В 4 из 10 столбцов присутствуют пропуски. 
* В столбцах Price Range и Cuisine Style очень большое количество пропусков. 
* По условию задания строки мы не удаляем, пробуем заменить.

Давайте пройдем по каждому из признаков, где есть пустые значения.

#### Признак Cuisine Style	

Пропущено 23.18% данных. 

Пропуски попробуем заполнить данными с Trip Advisor, но сделаем это позже, в секции формирования новых признаков (см. раздел [3.2 Обработка признаков](#3.2). Пропуски, которые заполнить не удастся реальными данными, заменим на значение 'Unknown'.

#### Признак Price Range

Пропущено 34.72% данных. 

Посмотрим на распределение признака, группировку по городу, рангу, продумаем варианты заполнения.

In [ ]:
# Предварительный просмотр данных
data['Price Range'].value_counts(normalize=True)

**Выводы по признаку:** 70% кухонь - средней ценовой категории

**Стратегия заполнения:**
1. Найти данные на внешних ресурсах
2. Значением моды.

In [ ]:
# Заполним пропуски модой
data['Price Range'].fillna(data['Price Range'].mode()[0], inplace = True)

#### Признак Number of Reviews

Пропущено 6.40% данных. 

Посмотрим на распределение признака, группировку по городу, продумаем варианты заполнения.

In [ ]:
# Предварительный просмотр данных
data['Number of Reviews'].hist()
data['Number of Reviews'].describe()

In [ ]:
# Посмотрим на распределение по городам
data.groupby(['City'])[
             'Number of Reviews'].agg(['max', 'min', 'mean', 'median'])

**Выводы по признаку:**
* Есть выбросы => сильное влияние на среднее по всему датасету.
* У 50% данных количество ревью от 7 до 105. Медиана - 28
* Есть зависимость среднего/медианы от города. 

**Стратегия заполнения:**
1. Найти данные на внешних ресурсах
2. Медианным значением в зависимости от города
3. Заполнить нулями

Отсутствие данного значения может быть важной информацией для модели. 

Поэтому давайте вынесем все пропуски в отдельный признак (number_of_rev_is_NAN).

In [ ]:
# Создаем новый признак
data['number_of_rev_is_NAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
# Заполняем пропуски медианой по городу
median_reviews = data.groupby(['City'])['Number of Reviews'].median() # series с медианами по городам
data['Number of Reviews'] = data.apply(lambda x: median_reviews.loc[x['City']] if pd.isna(
    x['Number of Reviews']) else x['Number of Reviews'], axis=1)

#### Признак Reviews

Пропусков очень мало. 

Обработку пустых значений добавила в раздел формирования новых признаков (см. раздел [3.2 Обработка признаков](#3.2)).

Посмотрим еще раз на информацию по пропускам.

In [ ]:
# Запускаем функцию вывода всех пустых значений
intitial_eda_checks(data, 0)

<a id="3.2"></a>
## 3.2. Обработка признаков

#### Поиск категориальных признаков для обработки
Для начала посмотрим, какие признаки могут быть категориальными.

In [ ]:
data.nunique(dropna=False)


Предварительно, категориальными выглядят:
* City
* Price Range
* Cuisine Style.

#### Чистка признака Restaurant_id

Признак Restaurant_id cодержит id_ перед номером, избавимся от префикса.

In [ ]:
data.Restaurant_id.sample(1)

In [ ]:
# Почистим формат колонок с ID
data.Restaurant_id = data.Restaurant_id.apply(lambda x: int(x[3:]))
data.ID_TA = data.ID_TA.apply(lambda x: int(x[1:]))
data.Restaurant_id.sample(1)

#### Подготовка данных с TripAdvisor

Данный с TripAdvisor (TA) были получены через решение https://apify.com/maxcopell/tripadvisor#api-usage на платформе APIFY. 

Залиты и добавлены в датафрейм data_ta.

С данных TA нам понадобятся точно:
* Список кухонь
* Колисество наград у ресторана
* Информация по расположению ресторана (широта и долгота).

Если позволит время, то дополинтельно можно использовать информацию для заполнения пропусков:
* Price Range
* Number of Reviews.


**Чистка данных**

Почистим пока ненужные колонки, сделаем преобразования датафрейма.

In [ ]:
# Удаляем ненужные колонки
# Для списка сертификатов будем использовать только имя сертификата
data_ta.drop([col for col in data_ta.columns if col.endswith(
    '/year')], axis=1, inplace=True)
data_ta.drop([col for col in data_ta.columns if col.startswith(
    'hours/')], axis=1, inplace=True)  # Время работы ресторана не используем
data_ta.drop(['address', 'phone', 'rankingPosition', 'type', 'webUrl', 'website', 'email',
              'isClosed', 'isLongClosed', 'rating'], axis=1, inplace=True)  # Доп список признаков, которые решила не использовать точно

data_ta.sample()

In [ ]:
# Переименуем колонку ID_TA для простоты последующего мержа
data_ta.rename(columns={"id": "ID_TA"}, inplace=True)
data_ta.sample()

In [ ]:
# Т.к. набор колонок у разных испточников данных разный, то отсортируем все полученные колонки по алфавиту
# для простоты работы и воспрозводимости кода
# переменная со списком отсортированных колонок
columns_list = list(data_ta.columns.sort_values())
data_ta = data_ta[columns_list]  # модифицируем датафрейм
data_ta.sample()

**Создадим новые признаки:**
1. awards_ta - список с наградами
2. awards_num - количество наград у ресторана
2. cuisine_styles_ta - список с кухнями

In [ ]:
# Создаем признак, который будет хранить все награды ресторана
data_ta['awards_ta'] = data_ta[data_ta.columns[1:12]].apply(
    lambda x: ', '.join(x.dropna().astype(str)),
    axis=1)  # Проходимся по колонкам с наградами, объединяем непустые значения в строку через запятую

data_ta['awards_ta'] = data_ta['awards_ta'].apply(
    lambda x: x.split(", "))  # создаем список наград для каждого ресторана

In [ ]:
# Создаем признак awards_num, который будет хранить количество наград у ресторана
len_cert_list = []

for i in range(0, len(data_ta)):
    if data_ta['awards_ta'][i][0] == '':  # если список наград пустой, то записываем 0
        len_cert_list.append(0)
    else:
        # если непустой, то записываем длину списка
        len_cert_list.append(len(data_ta['awards_ta'][i]))

data_ta['awards_num'] = len_cert_list  # добавляем признак

In [ ]:
# Создаем признак со списками кухонь, который будет хранить список кухонь дл] ресторана
data_ta['cuisine_styles_ta'] = data_ta[data_ta.columns[13:-9]].apply(
    lambda x: ', '.join(x.dropna().astype(str)),
    axis=1)  # Проходимся по колонкам с кухнями, объединяем непустые значения в строку через запятую

data_ta['cuisine_styles_ta'] = data_ta['cuisine_styles_ta'].apply(
    lambda x: x.split(", "))  # создаем список кухонь для каждого ресторана

Сформируем итоговый датафрейм с внешними данными, который будем использовать дальше для генерации признаков (data_ta_output).

In [ ]:
# Создаем датафрейм с колонками, которые хотим перенести в исходный датафрейм для модели data
data_ta_output = data_ta[['ID_TA', 'awards_num',
                          'cuisine_styles_ta', 'longitude', 'latitude']]
# удаляем дубликаты для ресторанов (такие есть) для корректного мержа
data_ta_output.drop_duplicates(subset=['ID_TA'], inplace=True)
data_ta_output.sample(1)

In [ ]:
# Смержим рабочий датафрейм с внешними данными из TA
data = pd.merge(data, data_ta_output, on="ID_TA",
                how="left")  # объединяем по ID_TA

data.sample(1)
data.info()  # Проверим, что количество строк осталось прежним

Внешние данные добавили. 

К сожалению, инструмент, который использовала, не предоставил исчерпывающую базу данных, поэтому по добавленным признакам есть пропуски (есть 41247 из 50000). Будем пропуски обрабатывать далее.

#### Признак AWARDS_NUM

Добавили новый признак с TA про количество наград.

Посмотрим на распределение признака awards_num.

In [ ]:
data.awards_num.hist(bins = 11)
data.awards_num.describe()

Посмотрим на пустые значения по этому признаку (17.5%) и заменим их на медиану, т.е. нули.

In [ ]:
# Вызовим функцию по просмотру NA значений
intitial_eda_checks(data, 0)

In [ ]:
# Заполняем медианой NA в awards_num
data.awards_num.fillna(data.awards_num.median(), inplace = True)

#### Признак CITY

Добавим в нашу выборку новые признаки по городам (данные из интеренета):
* country - страна, в которой находится город
* citizens - население города, чел
* restaurants_number_TA - количество ресторанов, участвующих в рейтинге (TripAdvisor)
* citizens_per_restaurant - количество горожан на один ресторан
* tourists_per_year - количество туристов, посетивших город в течение года, чел
* ttl_ppl_per_restaurants - (количество туристов + население города) / количество ресторанов

In [ ]:
# Мержим два датафрейма
data = pd.merge(data, cities_info, on="City", how="left")  # объединяем по City
data.sample(1)

In [ ]:
# Посмотрим, сколько уникальных городов
data.City.nunique()

В нашей выборке 31 город. Не очень много.
Для данного признака попробуем dymmy-кодирование.

In [ ]:
# Создадим признак с копией городов перед кодированием, т.к. изначальная колонка может быть полезной.
data['city_copies'] = data['City']

In [ ]:
# Используем One-Hot Encoding в pandas - get_dummies для кодирования городов.
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [ ]:
data.head(2)

#### Признак DISTANCE


На основе внешних данных созададим новый признак distance, который будет показывать расстояние от центра города до ресторана.

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    '''
    Функция принимает на вход координаты города и ресторана. 
    На выходе возвращает расстояние от центра города до ресторана.
    '''
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2) ** 2
    c = 2 * asin(sqrt(a))
    earth_radius = 6371  # in km
    return c * earth_radius

In [ ]:
# Создаем новый признак distance
data['distance'] = data.apply(lambda row: 
                                      haversine(lon1 = row['lon_c'], 
                                                lat1 = row['lat_c'],
                                                lon2 = row['longitude'], 
                                                lat2 = row['latitude']), 
                                      axis=1)

На предыдущих этапах мы выяснили, что не для всех ресторанов из внешних иточников была информацию о широте и долготе. 
Поэтому нужно не забыть заполнить пропуски. Заполним значением среднего.

In [ ]:
# Заполняем пропуски значением среднего по городу
mean_distance = data.groupby(['city_copies'])['distance'].mean()
data['distance'] = data.apply(lambda x: mean_distance.loc[x['city_copies']] if pd.isna(
    x['distance']) else x['distance'], axis=1)

# data['distance'].fillna(data['distance'].mean(), inplace = True)

Посмотрим, какое распределение признака получилось.

In [ ]:
data['distance'].hist()
data['distance'].describe()

#### Признак NUMBER OF REVIEWS

Число отзывов сильно влияет на ранг/рейтинг. При этом вр время EDA ниже было выявлено, что разброс значений очень большой.

Попробуем создать новый признак:
*  reviews_per_ttl_ppl - показывает сколько ревью приходится на суммарное 1000 людей (жители + туристы)

In [ ]:
# Создаем новый признак с использованием внешних данных по городам
data['reviews_per_ttl_ppl'] = data.apply(lambda row: (
    row['Number of Reviews']/(row['citizens']+row['tourists_per_year']))*1000, axis=1)

Посмотрим, какое распределение признака получили.

In [ ]:
data['reviews_per_ttl_ppl'].hist()

#### Признак PRICE RANGE

Посмотрим, какие значени содержит признак.

In [ ]:
data['Price Range'].unique()

По описанию 'Price Range' это - цены в ресторане. Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами.

Price_range можно разбить на числовой признак от 1 до 3:
* Низкий уровень цен - 1 
* Средний ценовой сегмент - 2 
* Высокий уровень цен - 3

Код ниже создаёт новый признак price_range_num.

In [ ]:
# Создаем словать с кодировкой значений в числовые
pricerange_dict = {"nan": 0, "$": 1, "$$ - $$$": 2, "$$$$": 3}
data['price_range_num'] = data['Price Range']
data['price_range_num'].replace(
    to_replace=pricerange_dict, inplace=True)  # заменяем значения в соответствии со словарем

In [ ]:
data.head(2)

#### Признак CUISINE STYLE

Посмотрим на содержание этого столбца. 

In [ ]:
# Количество уникальных значений
data['Cuisine Style'].nunique()

In [ ]:
# Примеры данных
data['Cuisine Style'].sample(5)

In [ ]:
# Тип данных
type(data['Cuisine Style'][0])

Почистим данные в столбце.

In [ ]:
def clean_name(str_val):
    """
    Преобразует строку с названиями кухонь в список [list] названий кухонь.
    На входе:
        - строковая переменная, содержащая названия кухонь.
    На выходе:
        - список [list] названий кухонь.
    """
    if pd.isna(str_val):
        return ['Unknown']
    str_val = str_val.strip('[]')  # Отбрасываем скобки
    str_val = str_val.replace("\'", '')  # Убираем кавычки '
    str_val = str_val.split(", ")  # Разбиваем строку по названиям кухонь
    return str_val

In [ ]:
# Применим ф-ию по чистке данных
data["Cuisine Style"] = data["Cuisine Style"].apply(clean_name)
data.sample(5)

Помним, что список кухонь содержит 23.18% пропусков. При чистке мы заменили их на "Unknown". Для данных, по которым нашли информацию на TA, сделаем замену.

In [ ]:
def cuisine_nan_replace(row):
    '''
    Функция на вход принимает строку датафрейма, проверяем ее значение.
    На выход возвращает или изначальное значение списка кухонь, или соотвествующий список с TA для тех кухонь, где указано Unknown.
    '''
    if row['Cuisine Style'][0] == 'Unknown':
        return row['cuisine_styles_ta']
    else:
        return row['Cuisine Style']

In [ ]:
# Заполняем данными с TA с помощью функции
data['Cuisine Style'] = data.apply(cuisine_nan_replace, axis=1)

In [ ]:
intitial_eda_checks(data, 0)

После заполнения данными с TA осталось 5% пропусков. Заполним их значением Unknown.

In [ ]:
# Заполняем значением Unknown
data['Cuisine Style'].fillna("Unknown", inplace = True)

Строки с Unknown типа str, а остальные - list. Сделаем преобразования.

In [ ]:
def clean_type(str_val):
    """
    Преобразует строку с Unknown названием кухни в список [list].
    На входе:
        - колонка, содержащая названия кухонь.
    На выходе:
        - список [list] названий кухонь.
    """
    if type(str_val) == str:
        return str_val.split()
    return str_val

In [ ]:
# Применяем функцию
data["Cuisine Style"] = data["Cuisine Style"].apply(clean_type)
data.sample(2)

Проанализируем, какое количество кухонь в среднем у ресторнов.

Среднее количество кухонь у ресторана - 2.6 (до заполнения данными с TA), 2.7 - после.

In [ ]:
# Посчитаем среднее количество кухонь у ресторана
count = 0

for i in range(0, len(data)):
    count += len(data['Cuisine Style'][i])

round(count/len(data), 1)

Также посмотрим, сколько уникальных кухонь.

Всего 125 уникальных кухонь (до заполнения с TA), 146 - после. 

In [ ]:
def data_explode(df, col, cnt=False):
    """
    Принимает на входе объект DataFrame df и 'имя' столбца col.
    Если cnt = True ("режим value_counts"):
        - возвращает объект series типа value_counts для столбца col.
    Если cnt = False ("режим DataFrame"):
        - возвращает объект DataFrame c "разъединёнными" элементами столбца col
    """
    df = df.explode(col)
    if cnt:
        return df[col].value_counts()
    return df

In [ ]:
# Посмотрим, сколько уникальных названий кухни, применив ф-ию.
cuisine_count = data.copy() # создадим копию датафрейма
data_explode(cuisine_count, 'Cuisine Style', cnt=True)

In [ ]:
# Создали отдельный df с кухнями в режиме explode
cuisine_count = data_explode(cuisine_count, 'Cuisine Style', cnt=False)
cuisine_count.sample(3)

In [ ]:
# Сразу посмотрим на распределение признака
(cuisine_count["Cuisine Style"].value_counts()).hist(bins = 10)

Видим, что всего представлено 146 уникльных кухонь. При этом подавляющее большинство (2/3) упоминается не так часто, но и есть особо популярные кухни. Можно подумать про объединение кухонь по частоте упоминания.

Пока сформируем признак cuisine_num, который будет показывать, сколько типов кухонь представлено у ресторана.

In [ ]:
# Добавляем признак cuisine_num
len_cuisines_list = []

for i in range(0, len(data)):
    if data['Cuisine Style'][i][0] == 'Unknown':
        len_cuisines_list.append(-1)  # -1 для пропуско
    elif data['Cuisine Style'][i][0] == '':
        len_cuisines_list.append(0)  # 0, где кухонь нет и на TA
    else:
        len_cuisines_list.append(len(data['Cuisine Style'][i]))

data['cuisine_num'] = len_cuisines_list

In [ ]:
# Посмотрим на распределение признака
data['cuisine_num'].describe()

Для ресторанов, для которых были пропуски и не заполнились данными с TA, заменим значение количества кухонь на медианное.

Такиех записей 2442.

Закомментировала, т.к. MAE при таком подходе хуже.

In [ ]:
# data['cuisine_num'].replace(-1, data['cuisine_num'].median(), inplace = True)

In [ ]:
data.sample(2)

По ходу проведения EDA было замечено, что определенные опции влияют на рейтинг/количество ревью.

У категории кухонь, отмеченных на TA, как Dietary Restrictions рейтинги выше. 
Можно отметить рестараны отдельным признаком, если такие опции у него имеются.
Какие опции включаем:
* Vegetarian Friendly
* Vegan Options
* Halal
* Kosher
* Gluten Free Options. 

Выделим в отдельный признак dietary_restrictions.

In [ ]:
def dietary_restrictions(row):
    """
    Функция на вход принимает строку датафрейма.
    Если в списке кухонь ресторана есть одна из кухонь списка спец. кухонь, то
        - возвращаем 1
        - иначе возвращаем 0.
    """
    dietary_restrictions = ['Vegetarian Friendly', 'Vegan Options',
                            'Gluten Free Options', 'Halal', 'Kosher']
    for i in dietary_restrictions:
        if i in row['Cuisine Style'] and i != '':
            return 1
    return 0

In [ ]:
# Создаем признак dietary_restrictions
data['dietary_restrictions'] = data.apply(dietary_restrictions, axis=1)

In [ ]:
data.sample(2)

И последний шаг - создание дамми-признаков для всех кухонь.

In [ ]:
# Используем MultiLabelBinarizer() для кодирования

s = data['Cuisine Style']
mlb = MultiLabelBinarizer()
cuisine_df = pd.DataFrame(mlb.fit_transform(
    s), columns=mlb.classes_, index=data.index)  # cсоздаем датафрейм с дамми кухнями

cuisine_df.head(3)

In [ ]:
# Смержим рабочий датафрейм с датафреймом дамми-кухонь
data = data.merge(cuisine_df, left_index=True, right_index=True)
data.info()
data.sample(2)

#### Признак REVIEWS

Посмотрим на reivews. Видим, что он содержит два ревью с датами ревью.
Мы можем вытащить несколько новых признака из дат:
* review_date: все даты ревью
* date_rev_1: дата первого ревью
* date_rev_2: дата второго ревью
* date_rev_delta: количество дней между оставленными ревью
* date_rev_from_max: количество дней от последнего отзыва до самого свежего отзыва в датасете.

In [ ]:
# Посмотрим, что содержится в столбце с ревью.
data.Reviews[1]

In [ ]:
# Тип данных - str
type(data.Reviews[1])

In [ ]:
# В тестовой выборке есть пустые значения, заменим их на строку, которая показывает, что ревью нет.
data['Reviews'].fillna('[[], []]', inplace=True)

In [ ]:
# Создадим новый признак review_date на основе патерна поиска дат.
pattern = re.compile('\d+\/\d+\/\d+')
data['review_date'] = data.Reviews.apply(pattern.findall)

data['review_date'].sample(5)

Видим, что review_date может содержать одну дату, две даты, три даты, ни одной даты.

Кодом ниже проверим, есть ли такие ревью, где дата содержалась в самом комментарии и создался список из трёх дат. Да, такие записи есть.
Применим к таким полям функцию, которая первое упоминание из комментариев почистит.

In [ ]:
# Напечатать даты, где более двух дат
for i in range (0, len(data)):
    if len(data.review_date[i]) > 2:
        print(i, len(data.review_date[i]))

In [ ]:
# Чистка данных, где в поле review_date попали даты-упоминания из комментариев отзыва.
data.review_date = data.review_date.apply(lambda x: [x[-2], x[-1]] if len(x) > 2 else x)

In [ ]:
# Посмотрим, сколько данных, где менее двух отзывов. 
count = 0
for i in range (0, len(data)):
    if len(data.review_date[i]) < 2:
        count +=1
count

Создадим признаки:
* date_rev_1: дата первого ревью
* date_rev_2: дата второго ревью
* date_rev_delta: количество дней между оставленными ревью
* date_rev_from_max: количество дней от последнего отзыва до самого свежего отзыва в датасете.
    
>     Информация с TA: Свежие отзывы имеют большую ценность, чем написанные давно. Они дают более точное представление о том, чего в данный момент стоит ожидать от компании. Это значит, что отзывы, которые были написаны давно (независимо от того, положительные они или отрицательные), имеют меньший вес при расчете рейтинга компании, чем отзыв, написанный недавно. Несмотря на то, что устаревшие отзывы не имеют такого же веса в рейтинге, они по-прежнему отображаются в разделе "Обзор" на странице каждого объекта в каталоге и в истории отзывов о компании.

In [ ]:
# Создаем новые признаки, сразу переводим в формат datetime64
data['date_rev_1'] = pd.to_datetime(data.review_date.apply(lambda x: x[0] if len(x) >=1 else None))
data['date_rev_2'] = pd.to_datetime(data.review_date.apply(lambda x: x[1] if len(x) >=2 else None))
data['date_rev_delta'] = (abs(data.date_rev_2-data.date_rev_1))/ np.timedelta64(1, "D")

In [ ]:
# Максимальная дата отзывы в датасете
date_max = data[['date_rev_1', 'date_rev_2']].max(axis =1).max()
date_max

In [ ]:
# Создаем новый признак про актуальность отзывов
data['date_rev_from_max'] = data.apply(lambda row: None if len(row.review_date) == 0  # если пустые значения, то Nan
                                       # если одна дата, то смотрим разницу с первым отзывом
                                       else (date_max-row.date_rev_1) if len(row.review_date) == 1
                                       else ((date_max-row.date_rev_2)), axis=1) / np.timedelta64(1, "D")  # если два отзыва, то берем второй отзыв

In [ ]:
data.sample(3)

#### Признак Restaurant_ID

Посмотрим на количество уникальный ID из 50 000 записей.

In [ ]:
# Количество уникальных ID
data.Restaurant_id.nunique()

Из 50000 записей только 13094 уникальных ID:
* 3807 - рестораны, представленные одним заведением
* Остальные 46193 - сетевые рестораны.

Создадим новый признак "in chain", который будет 
* 0 - если ресторан несетевой, 
* 1 - ресторан сетевой.

In [ ]:
# Найдем ID ресторанов, у которых в value_counts более одного ресторана, сохраним список
in_chain_index = data['Restaurant_id'].value_counts().loc[lambda x: x>1].index

In [ ]:
# запишем ID ресторанов, у кого value_counts > 1
data['in_chain'] = data['Restaurant_id'].apply(lambda x: 1 if x in in_chain_index else 0)

In [ ]:
# Посмотрим на получившиеся значения
data['in_chain'].value_counts()

<a id="4"></a>
# 4. EDA 

<a id="4.1"></a>

### 4.1 Распределение признаков

Функции для отрисовки графиков.

In [ ]:
def get_boxplot_2(column):
    """
    Функция для отрисовки коробочной диаграммы для нечисловых величин.
    На вход получаем список колонок для отрисовки. 
    Отрисовываем относительно целевой переменной Rating.
    """
    fig, ax = plt.subplots(figsize=(14, 4))
    sns.boxplot(x=column, y='Rating',
                data=data[data['sample'] == 1],
                ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot для ' + column)
    plt.show()

#### "Служебные признаки"

Признаки, которые не анализируем и которые удалим перед отправкой данных на обучение модели:
* URL_TA — URL страницы ресторана на TripAdvisor;
* ID_TA — идентификатор ресторана в базе данных TripAdvisor.

#### Признак Restaurant_id -> In_Chain

Мы сгенерировали признак "in chain", посмотрим на него.

In [ ]:
# Посмотрим на распределение признака
plt.rcParams['figure.figsize'] = (2,5)
data['in_chain'].hist(bins=2)

Посмотрим на распределение других признаков от того, является ли ресторан сетевым.

In [ ]:
# Посмотрим на распределение других признаков от того, является ли ресторан сетевым
fig, ax = plt.subplots(1, 3, figsize=(20, 10))
sns.scatterplot(data=data[data['sample'] == 1], x="in_chain", y="Rating", ax=ax[0])
sns.scatterplot(data=data[data['sample'] == 1], x="in_chain", y="Number of Reviews",ax=ax[1])
sns.scatterplot(data=data[data['sample'] == 1], x="in_chain", y="price_range_num",ax=ax[2])

Большинство ресторанов - сетевые. Наблюдается зависимость количества отзывов от того, сетевой ли ресторан. У сетевых ресторанов отзывов больше.

#### Признак Ranking

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

Чтобы скорректировать признак и сделать его более информативным создадим новый признак: 
* rank_per_ttl - показывает относительную позицию ранга ресторана к общему количеству рангов по городу. Предположила, что количетсво рангов по городу будет близко к общему количеству ресторанов по версии TA.
    Чем больше ранг - тем лучше. Чем меньше rank_per_ttl - тем лучше с позиции рейтинга.

In [ ]:
# Создаем признак rank_per_ttl
data['rank_per_ttl'] = data.apply(lambda x: x['Ranking']/x['restaurants_number_TA'], axis = 1)
data.sample(1)

In [ ]:
# Смотрим распределение
data['rank_per_ttl'].hist()

In [ ]:
# Посмотрим на распределение других признаков относительно нового признака
fig, ax = plt.subplots(1, 3, figsize=(20, 8))
sns.scatterplot(data=data[data['sample'] == 1], x="rank_per_ttl", y="Rating", ax=ax[0])
sns.scatterplot(data=data[data['sample'] == 1], x="Number of Reviews", y="rank_per_ttl",ax=ax[1])
sns.scatterplot(data=data[data['sample'] == 1], x="price_range_num", y="rank_per_ttl",ax=ax[2])

Для сравнения посмотрим на паспределение изначального Ranking к целевой переменной Rating и другими признаками.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 8))
sns.scatterplot(data=data[data['sample'] == 1], x="Ranking", y="Rating", ax=ax[0])
sns.scatterplot(data=data[data['sample'] == 1], x="Number of Reviews", y="Ranking",ax=ax[1])
sns.scatterplot(data=data[data['sample'] == 1], x="price_range_num", y="Ranking",ax=ax[2])

Видно, что новый признак имеет более выраженную зависимость с целевой переменной Rating.

**Графики:**
1. чем меньше ранг ресторана, тем чаще встречается более высокий рейтинг. Наличие корреляции с целевым признаком - хорошо для обучения модели.
2. чем меньше ранг ресторана, тем большее количество отзывов

Добавим признаки перемножением двух скоррелированных признаков с Ranking.

In [ ]:
# Добавление признаков
data["ranking_num_reviews"] = data["Ranking"] * data["Number of Reviews"]
data["ranking_num_cuisines"] = data["Ranking"] * data["cuisine_num"]

#### Признак Price Range

Будем использовать для анализа новый признак price_range_num, т.к. он полностью дублирует изначальный признак. Пропуски уже заполнены модой.

In [ ]:
plt.rcParams['figure.figsize'] = (2,5)
data['price_range_num'].hist(bins=3)
data['price_range_num'].describe()

**Вывод**: большая часть ресторанов средней ценовой категории. Самая немногочисленная часть - дорогие рестораны.

In [ ]:
# Посмотрим на распределение рейтингов по ценовым категориям.
get_boxplot_2('price_range_num')

In [ ]:
# Поссмотрим зависимость целевой переменной и ценовой категории на тестовой части выборки
fig, ax = plt.subplots(1, 2, figsize=(20, 8))
sns.scatterplot(data=data[data['sample'] == 1], x="price_range_num", y="Rating", ax=ax[0]);
sns.scatterplot(data=data[data['sample'] == 1], x="Number of Reviews", y="price_range_num", ax=ax[1]);

**Зависимость целевой переменной и ценовой категории**:
1. Как низкие, так и высокие рейтинги представлены во всех ценовых категориях
2. Наиболее разнообразное распределение рейтингов представлено во 2-ой ценовой категории. По 1-ой и 2-ой значения очень похожи.

Максимальная корреляция ценовой категории с количеством отзывов. Чем категория выше, тем больше отзывов.

#### Признак Number of Reviews

Посмотрим на распределение признака, на зависимость признака от целевой переменной и других переменных.

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Number of Reviews'].hist(bins=70)
df_train['Number of Reviews'].describe()

In [ ]:
sns.scatterplot(data=df_train, x="Number of Reviews", y="Rating")

Как и в случае с Ranking, количество отзывов очень отличается от ресторана/города (население, поток туристов).

Чтобы получить более информативную картину об отзывах ресторана, создадим новый признак:
* ttl_reviews_per_city - суммарное количество  ревью по городу из выборки
* reviews_perc_in_city_ttl - отношения количества ревью ресторана к суммарному количеству ревью по городу из выборки.

In [ ]:
# Создадим датафрейм, в который запишем суммы количества ревью по городам
reviews_sum = pd.DataFrame(data.groupby(['city_copies'])[
    'Number of Reviews'].sum().sort_values(ascending=False))
reviews_sum.rename(
    columns={"Number of Reviews": "ttl_reviews_per_city"}, inplace=True)
reviews_sum

In [ ]:
# Смержим созданный датафрейм с исходным датафреймам по городу
data = pd.merge(data, reviews_sum, on="city_copies", how="left")
data.sample(1)
data.info()

In [ ]:
# Создаем новый признак reviews_perc_in_city_ttl
data['reviews_perc_in_city_ttl'] = data.apply(lambda x: x['Number of Reviews']/x['ttl_reviews_per_city'], axis = 1)
data.sample(1)

По распределению изначальных данных количества ревью видим, что есть выбросы, устраним их.

In [ ]:
def iqr_analysis(series, mode = False):
    """
    Функция выводит инфорамцию о границах выборосов для признака.
    Если mode = True, возвращается верхняя и нижняя границы выбросов.
    """
    IQR = series.quantile(0.75) - series.quantile(0.25)
    perc25 = series.quantile(0.25)
    perc75 = series.quantile(0.75)

    f=perc25 - 1.5*IQR
    l=perc75 + 1.5*IQR
    
    if mode: 
        return f, l
    
    print(
        '25-й перцентиль: {},'.format(perc25),
        '75-й перцентиль: {},'.format(perc75),
        "IQR: {}, ".format(IQR),
        "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))

In [ ]:
# Используем функцию для расчета границ выбросов для всех данных
iqr_analysis(data['Number of Reviews'])

In [ ]:
# Посмотрим на границы выбросов по городам
cols = ["lower_border", "higher_border"]
lst = []

for x in (df_train['City'].value_counts()).index:
    lst.append(iqr_analysis(
        df_train['Number of Reviews'][df_train['City'] == x], mode=True))

reviews_IQ = pd.DataFrame(lst, columns=cols)
reviews_IQ['city'] = df_train['City'].value_counts().index

display(reviews_IQ)
print('Максимальное значение среди городов по верхней границе выбросов:', reviews_IQ.higher_border.max())

**Выводы**:
* Выглядит так, что в количестве отзывов есть выбросы.
* Границы выбросов варьируются от города к городу.

In [ ]:
# Посмотрим на распределение признака до максимальной границы выбраса
df_train[df_train['Number of Reviews'] < reviews_IQ.higher_border.max()]['Number of Reviews'].hist(bins=70)


Заменяем выбросы на 840.

In [ ]:
# Количество выбросов при границе в 840
len(data[data['Number of Reviews'] > 840]['Number of Reviews'])

In [ ]:
# Заменим выбросы в датафрейме data на максимальное пограничное знаечение признака
print('Будет заменено записей:', len(data[data['Number of Reviews'] > 840]['Number of Reviews']))
data['Number of Reviews'] = data['Number of Reviews'].apply(lambda x: 840 if x >=840 else x)

In [ ]:
sns.scatterplot(data=data[data['sample'] == 1], x="Number of Reviews", y="Rating")

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
sns.boxplot(x='Rating', y='Number of Reviews',
                data=data[data['sample'] == 1],
                ax=ax)
plt.xticks(rotation=45)
ax.set_title('Boxplot для Reviews/Rating')
plt.show()

#### Признак City

Посмотрим на распределение признака, на зависимость признака от целевой переменной и других переменных.

In [ ]:
# Посмотрим на распределению ресторанов по городам в %
display(pd.DataFrame(
        data['city_copies'].value_counts(normalize=True)*100))

In [ ]:
# Посмотрим на распределение рейтингов по городам.
get_boxplot_2('city_copies')

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
sns.boxplot(x='city_copies', y='Number of Reviews',
                data=data[data['sample'] == 1],
                ax=ax)
plt.xticks(rotation=45)
ax.set_title('Boxplot для city_copies')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
sns.boxplot(x='city_copies', y='price_range_num',
                data=data[data['sample'] == 1],
                ax=ax)
plt.xticks(rotation=45)
ax.set_title('Boxplot для city_copies')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
sns.boxplot(x='city_copies', y='date_rev_delta',
                data=data[data['sample'] == 1],
                ax=ax)
plt.xticks(rotation=45)
ax.set_title('Boxplot для city_copies')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
sns.boxplot(x='city_copies', y='Ranking',
                data=data[data['sample'] == 1],
                ax=ax)
plt.xticks(rotation=45)
ax.set_title('Boxplot для city_copies')
plt.show()

**Выводы по графику**:
1. Зависимость с целевой переменной:
    * Медиана по всем городам совпадает (4), кроме Милана
    * Кухня Милана самая низко-оцениваемая
    * Распределение рейтингов двух типов: а) от 3.5-4.5 с длинным хвостом до 2 b) 4-4.5 с коротким хвостом до 3.5.

    Нельзя формировать новые признаки на базе целевой переменной, но хорошо бы найти закономерность в разбиении городов на 2-3 группы.

2. Распределение рангов и городов тоже очень отличается. Выглядит так, что ранг завязан на количество ресторанов в городе. Чем больше ресторанов, тем размашистее распределение рангов между городами.
3. Дельта между двумя последними отзывами по городам практически идентична по своему распределению.
4. Количество ревью имеет больший размах для более туристических городов.

**Идеи по генерации новых признаков**:
1. Посмотреть доп. признаки по городам: население, общее количество ресторанов, ранг/общее количество рестаранов, количество туристов в год.
2. Найти признаки, по которым распределение с рейтингом будет иметь похожее на город/рейтинг для объединения в группы.

Посмотрим, как новые признаки связаны со старыми.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 10))
sns.scatterplot(data=data[data['sample'] == 1], x="citizens", y="Ranking", ax=ax[0])
sns.scatterplot(data=data[data['sample'] == 1], x="tourists_per_year", y="Number of Reviews",ax=ax[1])
sns.scatterplot(data=data[data['sample'] == 1], x="citizens", y="restaurants_number_TA",ax=ax[2])

In [ ]:
get_boxplot_2('country')

#### Признак Reviews

Посмотрим на распределение сгенерированного признака.

In [ ]:
data['date_rev_delta'].hist()
data['date_rev_delta'].describe()

Явно есть выбросы. Обработаем их.

In [ ]:
# Используем функцию для расчета границ выбросов для всех данных
iqr_analysis(data['date_rev_delta'])

In [ ]:
# Посмотрим, сколько записей содержит отзывы, где дельта между отзывами более года. Это чуть больше верхней границы по IQR
data[data['date_rev_delta']>365]['date_rev_delta'].count()

In [ ]:
# Заменим значение на 365*3 для выбросов (выбрано экспериментально)
data['date_rev_delta'] = data['date_rev_delta'].apply(lambda x: 1095 if x >1095 else x)

Обработаем пропуски.

In [ ]:
# Заменим NA на среднее
data['date_rev_delta'].fillna(data['date_rev_delta'].mean(), inplace = True)

Посмотрим, как изменилось распределение признака после замен.


In [ ]:
data['date_rev_delta'].hist()
data['date_rev_delta'].describe()

In [ ]:
data['date_rev_from_max'].hist()
data['date_rev_from_max'].describe()

In [ ]:
# Используем функцию для расчета границ выбросов для всех данных
iqr_analysis(data['date_rev_from_max'])

In [ ]:
# Посмотрим, сколько записей содержат данные, где признак больше верхней границы
data[data['date_rev_from_max']>1132]['date_rev_from_max'].count()

In [ ]:
# Заменим значение на 1132 для выбросов
data['date_rev_from_max'] = data['date_rev_from_max'].apply(lambda x: 1132 if x >1132 else x)

In [ ]:
# Пропуски заменим средним
data['date_rev_from_max'].fillna(data['date_rev_from_max'].mean(), inplace = True)

In [ ]:
data.sample()

#### Признак Cuisines

Посмотрим на распределение признака.

In [ ]:
data['cuisine_num'].value_counts(ascending=True).plot(kind='barh')
data['cuisine_num'].describe()

In [ ]:
# Поссмотрим зависимость целевой переменной и количеству кухонь на тестовой части выборки

fig, ax = plt.subplots(1, 2, figsize=(20, 8))
sns.scatterplot(data=data[data['sample'] == 1], x="cuisine_num", y="Rating", ax=ax[0]);
sns.scatterplot(data=data[data['sample'] == 1], x="cuisine_num", y="Number of Reviews", ax=ax[1]);

In [ ]:
cuisine_count.sample()

In [ ]:
# Посмотрим на распределение рейтингов по кухням на датафрейме, где отработал explode (cuisine_count)
fig, ax = plt.subplots(figsize=(60, 4))
sns.boxplot(x='Cuisine Style', y='Rating',
                data=cuisine_count[cuisine_count['sample'] == 1],
                ax=ax)
plt.xticks(rotation=45)
ax.set_title('Boxplot для cuisines')
plt.show()

**Наблюдение**: у категории кухонь, отмеченных на TA, как Dietary Restrictions рейтинги выше. 
Можно отметить рестараны отдельным признаком, если такие опции у него имеются.
Какие опции включаем:
* Vegetarian Friendly
* Vegan Options
* Halal
* Kosher
* Gluten Free Options. 

<a id="4.2"></a>

### 4.2  Распределение целевой переменной Rating

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')
df_train['Rating'].describe()

In [ ]:
df_train['Rating'].unique()

Рейтинги распределены от 1 до 5 с шагом в 0.5.

<a id="4.3"></a>

### 4.3 Корреляция признаков
Проведем корреляционный анализ.

Удалим из анализа:
* Категориальные признаки (кухни, города)
* sample (служебный признак).

In [ ]:
# Сфорсмруем список признаков, которые исключаем из корреляционного анализа
cols_to_drop = ['sample', 'city_copies',  'City_Amsterdam',  'City_Athens',  'City_Barcelona', 'City_Berlin',  'City_Bratislava',  'City_Brussels',  'City_Budapest',  'City_Copenhagen',  'City_Dublin',  'City_Edinburgh',  'City_Geneva',  'City_Hamburg',  'City_Helsinki', 'City_Krakow',  'City_Lisbon',  'City_Ljubljana',  'City_London',  'City_Luxembourg',  'City_Lyon',  'City_Madrid',  'City_Milan',  'City_Munich',  'City_Oporto',  'City_Oslo',  'City_Paris', 'City_Prague',  'City_Rome',  'City_Stockholm',  'City_Vienna',  'City_Warsaw',  'City_Zurich',  'City_nan', '',  'Afghani',  'African',  'Albanian',  'American',  'Arabic',  'Argentinean', 'Armenian',  'Asian',  'Australian',  'Austrian',  'Azerbaijani',  'Balti',  'Bangladeshi',  'Bar',  'Barbecue',  'Beer restaurants',  'Belgian',  'Brazilian',  'Brew Pub',  'British',  'Burmese',  'Cafe',  'Cajun & Creole',  'Cambodian',  'Campania',  'Canadian',  'Caribbean',  'Catalan',  'Caucasian',  'Central American',  'Central Asian',  'Central European',  'Central-Italian',  'Chilean',  'Chinese',  'Colombian',  'Contemporary',  'Croatian',  'Cuban',  'Czech',  'Danish',  'Deli',  'Delicatessen',  'Diner',  'Dining bars',  'Dutch',  'Eastern European',  'Ecuadorean',  'Egyptian',  'Emilian',  'Ethiopian',
                'European', 'Fast Food',  'Filipino',  'French',  'Fruit parlours',  'Fujian',  'Fusion',  'Gastropub',  'Georgian',  'German',  'Gluten Free Options',  'Greek',  'Grill',  'Halal',  'Hawaiian',  'Healthy',  'Hungarian',  'Indian',  'Indonesian',  'International',  'Irish',  'Israeli',  'Italian',  'Jamaican',  'Japanese',  'Japanese Fusion',  'Korean',  'Kosher',  'Latin',  'Latvian',  'Lazio',  'Lebanese',  'Lombard',  'Malaysian',  'Mediterranean',  'Mexican',  'Middle Eastern',  'Minority Chinese',  'Mongolian',  'Moroccan',  'Native American',  'Neapolitan',  'Nepali',  'New Zealand',  'Northern-Italian',  'Norwegian',  'Pakistani',  'Persian',  'Peruvian',  'Pizza',  'Polish',  'Polynesian',  'Portuguese',  'Pub',  'Romagna',  'Romana',  'Romanian',  'Russian',  'Salvadoran',  'Sardinian',  'Scandinavian',  'Scottish',  'Seafood',  'Sicilian',  'Singaporean',  'Slovenian',  'Soups',  'South American',  'Southern-Italian',  'Southwestern',  'Spanish',  'Sri Lankan',  'Steakhouse',  'Street Food',  'Sushi',  'Swedish',  'Swiss',  'Taiwanese',  'Thai',  'Tibetan',  'Tunisian',  'Turkish',  'Tuscan',  'Ukrainian',  'Uzbek',  'Vegan Options',  'Vegetarian Friendly',  'Venezuelan',  'Vietnamese',  'Welsh',  'Wine Bar',  'Xinjiang',  'Yunnan',  'Unknown']

In [ ]:
# Построим матрицу корреляций
plt.figure(figsize=(30, 15))
heatmap = sns.heatmap(data[data['sample'] == 1].drop(
    cols_to_drop, axis=1).corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Матрица корреляций', fontdict={'fontsize': 18}, pad=12)

In [ ]:
#.Подсветим те значения, где коэффициент корреляции больше заданного порога
plt.figure(figsize=(30, 15))
heatmap = sns.heatmap(abs(data[data['sample'] == 1].drop(
    cols_to_drop, axis=1).corr()) > 0.8, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Матрица корреляций, где корреляция > 0.8', fontdict={'fontsize':18}, pad=12);

In [ ]:
# Посмотрим на корреляцию признаков с целевой переменной Rating, отсортируем
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(data[data['sample'] == 1].drop(
    cols_to_drop, axis=1).corr()[['Rating']].sort_values(by='Rating', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Корреляция признаков с Rating', fontdict={'fontsize':18}, pad=16);

Сформируем список признаков, которые коллинеарны.

Для этого выставим критерий наличия корреляции больше 0.8 или -0.8.

In [ ]:
# Сформируем сет со скоррелированными признаками
correlated_features = set()
# Удаляем целевую переменную из матрицы коррелиций, тк корреляция с ней, - хорошо для модели
correlation_matrix = data[data['sample'] == 1].drop(
    ['Rating', 'sample'], axis=1).corr()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[j]
            correlated_features.add(colname)

print('Список скоррелированных признаков на удаление из обучения модели:', correlated_features)

<a id="4.4"></a>

### 4.4 Поиск статистически значимых различий с помощью теста Стьюдента

Графики являются лишь вспомогательным инструментом, настоящую значимость различий может помочь распознать статистика. 

Проверим, есть ли статистическая разница в распределении оценок по номинативным признакам, с помощью **теста Стьюдента**. Проверим нулевую гипотезу о том, что распределения рейтинга по различным параметрам неразличимы.

Анализ будем проводить для колонок, которые номинативные по типу данных, но и для колонок-шпионов, которые количественные, но обозначают номинативный признак (принадлежность к городу, типу кухни).

In [ ]:
def get_stat_dif_2(column):
    """ 
    Поиск статистически значимых различий для колонки с помощью теста Стьюдента.
    """
    cols = data[data['sample'] == 1].loc[:, column].value_counts().index[:]
    combinations_all = list(combinations(cols, 2))
    # Тест проводим на изначальном наборе данных без NA значений для целевого столбца, столбца с признаком, дополнительно исключив 0 для оценок
    stud_stat = data[data['sample'] == 1]
    for comb in combinations_all:
        if ttest_ind(stud_stat.loc[data[data['sample'] == 1].loc[:, column] == comb[0], 'Rating'],
                     stud_stat.loc[data[data['sample'] == 1].loc[:, column] == comb[1], 'Rating']).pvalue <= 0.05/len(combinations_all):  # учли поправку Бонферони
            # print('Найдены статистически значимые различия для колонки', column)
            pass
        else:
            return column 
            break

In [ ]:
# Сформируем сет для статистически незначимых признаков
to_remove_features = set()

# Проходим по колонкам, которые исключали из корреляционного анализа
for column in cols_to_drop: 
    to_remove_features.add(get_stat_dif_2(column))
    
print('\n Список признаков на удаление из обучения модели:', to_remove_features)

Объединяем списки колонок на удаление из анализа выше.

In [ ]:
# Формируем сет, конвертируем в список, удаляем NAN
drop_features = correlated_features.union(to_remove_features)
drop_features = list(drop_features)
drop_features.remove(None)

 <a id="5"></a>

# 5. DATA PREPROCESSING
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# На всякий случай, заново подгружаем данные
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'sample_submission.csv')
cities_info = pd.read_csv('/kaggle/input/citiesdata-2/cities_data.csv')

# Создадим датафрейм data_ta из данных, которые собрали с TripAdvisor
# Открываем каждый файл и добавляем его в data_ta
data_ta = pd.DataFrame()

for dirname, _, filenames in os.walk('/kaggle/input/data-ta/'):
    for filename in filenames:
        temp = pd.read_csv(os.path.join(dirname, filename))
        data_ta = pd.concat([data_ta, temp], ignore_index = True)

df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
def preproc_data(df_input, data_ta_input):
    '''Включены функции и операции по предобработке данных для модели.'''

    df_output = df_input.copy()
    data_ta = data_ta_input.copy()

    #################### 1. Предобработка ##############################################################
    # Соберем здесь используемые функции, обработку данных.

    def haversine(lon1, lat1, lon2, lat2):
        '''
        Функция принимает на вход координаты города и ресторана. 
        На выходе возвращает расстояние от центра города до ресторана.
        '''
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2) ** 2
        c = 2 * asin(sqrt(a))
        earth_radius = 6371  # in km
        return c * earth_radius

    def clean_name(str_val):
        """
        Преобразует строку с названиями кухонь в список [list] названий кухонь.
        На входе:
            - строковая переменная, содержащая названия кухонь.
        На выходе:
            - список [list] названий кухонь.
        """
        if pd.isna(str_val):
            return ['Unknown']
        str_val = str_val.strip('[]')  # Отбрасываем скобки
        str_val = str_val.replace("\'", '')  # Убираем кавычки '
        str_val = str_val.split(", ")  # Разбиваем строку по названиям кухонь
        return str_val

    def cuisine_nan_replace(row):
        '''
        Функция на вход принимает строку датафрейма, проверяем ее значение.
        На выход возвращает или изначальное значение списка кухонь, или соотвествующий список с TA для тех кухонь, где указано Unknown.
        '''
        if row['Cuisine Style'][0] == 'Unknown':
            return row['cuisine_styles_ta']
        else:
            return row['Cuisine Style']

    def clean_type(str_val):
        """
        Преобразует строку с Unknown названием кухни в список [list].
        На входе:
            - колонка, содержащая названия кухонь.
        На выходе:
            - список [list] названий кухонь.
        """
        if type(str_val) == str:
            return str_val.split()
        return str_val

    def dietary_restrictions(row):
        """
        Функция на вход принимает строку датафрейма.
        Если в списке кухонь ресторана есть одна из кухонь списка спец. кухонь, то
            - возвращаем 1
            - иначе возвращаем 0.
        """
        dietary_restrictions = ['Vegetarian Friendly', 'Vegan Options',
                                'Gluten Free Options', 'Halal', 'Kosher']
        for i in dietary_restrictions:
            if i in row['Cuisine Style'] and i != '':
                return 1
        return 0

    def get_stat_dif_2(column):
        """ 
        Поиск статистически значимых различий для колонки с помощью теста Стьюдента.
        """
        cols = df_output[df_output['sample'] == 1].loc[:, column].value_counts().index[:]
        combinations_all = list(combinations(cols, 2))
        # Тест проводим на изначальном наборе данных без NA значений для целевого столбца, столбца с признаком, дополнительно исключив 0 для оценок
        stud_stat = df_output[df_output['sample'] == 1]
        for comb in combinations_all:
            if ttest_ind(stud_stat.loc[df_output[df_output['sample'] == 1].loc[:, column] == comb[0], 'Rating'],
                         stud_stat.loc[df_output[df_output['sample'] == 1].loc[:, column] == comb[1], 'Rating']).pvalue <= 0.05/len(combinations_all):  # учли поправку Бонферони
                # print('Найдены статистически значимые различия для колонки', column)
                pass
            else:
                return column
                break

    # Почистим формат колонок с ID (ID_TA, Restaurant_id), избавимся от префиксов
    df_output.Restaurant_id = df_output.Restaurant_id.apply(
        lambda x: int(x[3:]))
    df_output.ID_TA = df_output.ID_TA.apply(lambda x: int(x[1:]))

    # Обработка внешних данных с TA
    # Удаляем ненужные колонки
    # Для списка сертификатов будем использовать только имя сертификата
    data_ta.drop([col for col in data_ta.columns if col.endswith(
        '/year')], axis=1, inplace=True)
    data_ta.drop([col for col in data_ta.columns if col.startswith(
        'hours/')], axis=1, inplace=True)  # Время работы ресторана не используем
    data_ta.drop(['address', 'phone', 'rankingPosition', 'type', 'webUrl', 'website', 'email',
                  'isClosed', 'isLongClosed', 'rating'], axis=1, inplace=True)  # Доп список признаков, которые решила не использовать точно
    # Переименуем колонку ID_TA для простоты последующего мержа
    data_ta.rename(columns={"id": "ID_TA"}, inplace=True)
    # Т.к. набор колонок у разных испточников данных разный, то отсортируем все полученные колонки по алфавиту
    # для простоты работы и воспрозводимости кода
    # переменная со списком отсортированных колонок
    columns_list = list(data_ta.columns.sort_values())
    data_ta = data_ta[columns_list]  # модифицируем датафрейм

    # Создаем признак, который будет хранить все награды ресторана
    data_ta['awards_ta'] = data_ta[data_ta.columns[1:12]].apply(
        lambda x: ', '.join(x.dropna().astype(str)),
        axis=1)  # Проходимся по колонкам с наградами, объединяем непустые значения в строку через запятую
    data_ta['awards_ta'] = data_ta['awards_ta'].apply(
        lambda x: x.split(", "))  # создаем список наград для каждого ресторана

    # Создаем признак awards_num, который будет хранить количество наград у ресторана
    len_cert_list = []
    for i in range(0, len(data_ta)):
        if data_ta['awards_ta'][i][0] == '':  # если список наград пустой, то записываем 0
            len_cert_list.append(0)
        else:
            # если непустой, то записываем длину списка
            len_cert_list.append(len(data_ta['awards_ta'][i]))
    data_ta['awards_num'] = len_cert_list  # добавляем признак

    # Создаем признак со списками кухонь, который будет хранить список кухонь дл] ресторана
    data_ta['cuisine_styles_ta'] = data_ta[data_ta.columns[13:-9]].apply(
        lambda x: ', '.join(x.dropna().astype(str)),
        axis=1)  # Проходимся по колонкам с кухнями, объединяем непустые значения в строку через запятую
    data_ta['cuisine_styles_ta'] = data_ta['cuisine_styles_ta'].apply(
        lambda x: x.split(", "))  # создаем список кухонь для каждого ресторана

    # Создаем датафрейм с колонками, которые хотим перенести в исходный датафрейм для модели data
    data_ta_output = data_ta[['ID_TA', 'awards_num',
                              'cuisine_styles_ta', 'longitude', 'latitude']]
    # Удаляем дубликаты для ресторанов (такие есть) для корректного мержа
    data_ta_output.drop_duplicates(subset=['ID_TA'], inplace=True)

    # Смержим рабочий датафрейм с внешними данными из TA
    df_output = pd.merge(df_output, data_ta_output, on="ID_TA",
                         how="left")  # объединяем по ID_TA

    # Мержим рабочий датафрейм с внешними данными по городам
    df_output = pd.merge(df_output, cities_info, on="City",
                         how="left")  # объединяем по City

    # Создадим признак с копией городов перед дамми-кодированием, т.к. изначальная колонка может быть полезной.
    df_output['city_copies'] = df_output['City']

    # Применим ф-ию по чистке данных для кухонь
    df_output["Cuisine Style"] = df_output["Cuisine Style"].apply(clean_name)

    # Создадим датафрейм, в который запишем суммы количества ревью по городам
    reviews_sum = pd.DataFrame(df_output.groupby(['city_copies'])[
        'Number of Reviews'].sum().sort_values(ascending=False))
    reviews_sum.rename(
        columns={"Number of Reviews": "ttl_reviews_per_city"}, inplace=True)
    # Смержим созданный датафрейм с исходным датафреймам по городу
    df_output = pd.merge(df_output, reviews_sum, on="city_copies", how="left")

    # ################### 2. NAN ##############################################################
    # Заполним пропуски Price Range модой
    df_output['Price Range'].fillna(
        df_output['Price Range'].mode()[0], inplace=True)

    # Заполняем медианой NA в awards_num
    df_output.awards_num.fillna(df_output.awards_num.median(), inplace=True)

    # Заполняем данными про типы кухонь с TA с помощью функции
    df_output['Cuisine Style'] = df_output.apply(cuisine_nan_replace, axis=1)
    # Заполняем значением Unknown, тк не все данные были на TA
    df_output['Cuisine Style'].fillna("Unknown", inplace=True)
    # Строки с Unknown типа str, а остальные - list. Сделаем преобразования.
    df_output["Cuisine Style"] = df_output["Cuisine Style"].apply(clean_type)

    # Создаем новый признак до заполнения пропусков
    df_output['number_of_rev_is_NAN'] = pd.isna(
        df_output['Number of Reviews']).astype('uint8')
    # Заполняем пропуски Number of Reviews медианой по городу
    median_reviews = df_output.groupby(
        ['City'])['Number of Reviews'].median()  # series с медианами по городам
    df_output['Number of Reviews'] = df_output.apply(lambda x: median_reviews.loc[x['City']] if pd.isna(
        x['Number of Reviews']) else x['Number of Reviews'], axis=1)

    # Заменим выбросы в датафрейме data на максимальное пограничное знаечение признака
    df_output['Number of Reviews'] = df_output['Number of Reviews'].apply(
        lambda x: 840 if x >= 840 else x)

    # В тестовой выборке есть пустые значения, заменим их на строку, которая показывает, что ревью нет.
    df_output['Reviews'].fillna('[[], []]', inplace=True)

    # ################### 3. Encoding ##############################################################
    # Используем One-Hot Encoding в pandas - get_dummies для кодирования городов.
    df_output = pd.get_dummies(df_output, columns=['City', ], dummy_na=True)

    # Используем MultiLabelBinarizer() для кодирования cписка кухонь
    s = df_output['Cuisine Style']
    mlb = MultiLabelBinarizer()
    cuisine_df = pd.DataFrame(mlb.fit_transform(
        s), columns=mlb.classes_, index=data.index)  # cсоздаем датафрейм с дамми кухнями
    # Смержим рабочий датафрейм с датафреймом дамми-кухонь
    df_output = df_output.merge(cuisine_df, left_index=True, right_index=True)

    # ################### 4. Feature Engineering ####################################################
    # Создаем новый признак distance, который будет показывать расстояние от центра города до ресторана.
    df_output['distance'] = df_output.apply(lambda row:
                                            haversine(lon1=row['lon_c'],
                                                      lat1=row['lat_c'],
                                                      lon2=row['longitude'],
                                                      lat2=row['latitude']),
                                            axis=1)
    # Заполняем пропуски значением среднего по городу
    mean_distance = df_output.groupby(['city_copies'])['distance'].mean()
    df_output['distance'] = df_output.apply(lambda x: mean_distance.loc[x['city_copies']] if pd.isna(
        x['distance']) else x['distance'], axis=1)

    # Создаем новый признак с использованием внешних данных по городам
    # reviews_per_ttl_ppl - показывает сколько ревью приходится на суммарное 1000 людей (жители + туристы)
    df_output['reviews_per_ttl_ppl'] = df_output.apply(lambda row: (
        row['Number of Reviews']/(row['citizens']+row['tourists_per_year']))*1000, axis=1)

    # Price Range - переведем в цифровые значения
    pricerange_dict = {"nan": 0, "$": 1, "$$ - $$$": 2, "$$$$": 3}
    df_output['price_range_num'] = df_output['Price Range']
    df_output['price_range_num'].replace(
        to_replace=pricerange_dict, inplace=True)  # заменяем значения в соответствии со словарем

    # Добавляем признак cuisine_num
    len_cuisines_list = []
    for i in range(0, len(df_output)):
        if df_output['Cuisine Style'][i][0] == 'Unknown':
            len_cuisines_list.append(-1)  # -1 для пропуско
        elif df_output['Cuisine Style'][i][0] == '':
            len_cuisines_list.append(0)  # 0, где кухонь нет и на TA
        else:
            len_cuisines_list.append(len(df_output['Cuisine Style'][i]))
    df_output['cuisine_num'] = len_cuisines_list

    # Создаем признак dietary_restrictions
    df_output['dietary_restrictions'] = df_output.apply(
        dietary_restrictions, axis=1)

    # Создадим новый признак review_date на основе патерна поиска дат.
    pattern = re.compile('\d+\/\d+\/\d+')
    df_output['review_date'] = df_output.Reviews.apply(pattern.findall)
    # Чистка данных, где в поле review_date попали даты-упоминания из комментариев отзыва.
    df_output.review_date = df_output.review_date.apply(
        lambda x: [x[-2], x[-1]] if len(x) > 2 else x)

    # Создаем новые признаки, сразу переводим в формат datetime64
    df_output['date_rev_1'] = pd.to_datetime(
        df_output.review_date.apply(lambda x: x[0] if len(x) >= 1 else None))
    df_output['date_rev_2'] = pd.to_datetime(
        df_output.review_date.apply(lambda x: x[1] if len(x) >= 2 else None))
    df_output['date_rev_delta'] = (
        abs(df_output.date_rev_2-df_output.date_rev_1)) / np.timedelta64(1, "D")

    # Создаем новый признак про актуальность отзывов
    date_max = df_output[['date_rev_1', 'date_rev_2']].max(axis =1).max()
    df_output['date_rev_from_max'] = df_output.apply(lambda row: None if len(row.review_date) == 0  # если пустые значения, то Nan
                                                     # если одна дата, то смотрим разницу с первым отзывом
                                                     else (date_max-row.date_rev_1) if len(row.review_date) == 1
                                                     else ((date_max-row.date_rev_2)), axis=1) / np.timedelta64(1, "D")  # если два отзыва, то берем второй отзыв

    # Заменим значение на 365*3 для выбросов (выбрано экспериментально)
    df_output['date_rev_delta'] = df_output['date_rev_delta'].apply(
        lambda x: 1095 if x > 1095 else x)
    # Заменим NA на среднее
    df_output['date_rev_delta'].fillna(
        df_output['date_rev_delta'].mean(), inplace=True)

    # Заменим значение на 1132 для выбросов (верхняя граница по IQR)
    df_output['date_rev_from_max'] = df_output['date_rev_from_max'].apply(
        lambda x: 1132 if x > 1132 else x)
    # Пропуски заменим средним
    df_output['date_rev_from_max'].fillna(
        df_output['date_rev_from_max'].mean(), inplace=True)

    # Создадим признак о том, что ресторан сетевой
    # Сначала найдем ID ресторанов, у которых в value_counts более одного ресторана, сохраним список
    in_chain_index = df_output['Restaurant_id'].value_counts().loc[lambda x: x>1].index
    df_output['in_chain'] = df_output['Restaurant_id'].apply(
        lambda x: 1 if x in in_chain_index else 0)

    # Создаем признак rank_per_ttl
    # rank_per_ttl - показывает относительную позицию ранга ресторана к общему количеству рангов по городу.
    df_output['rank_per_ttl'] = df_output.apply(
        lambda x: x['Ranking']/x['restaurants_number_TA'], axis=1)

    # Добавление признаков перемножением
    df_output["ranking_num_reviews"] = df_output["Ranking"] * \
        df_output["Number of Reviews"]
    df_output["ranking_num_cuisines"] = df_output["Ranking"] * \
        df_output["cuisine_num"]

    # Создаем новый признак reviews_perc_in_city_ttl
    # reviews_perc_in_city_ttl - отношения количества ревью ресторана к суммарному количеству ревью по городу из выборки
    df_output['reviews_perc_in_city_ttl'] = df_output.apply(
        lambda x: x['Number of Reviews']/x['ttl_reviews_per_city'], axis=1)

    # ################### 5. Clean ####################################################
    # Удаляем признаки, которые не отобрали для модели во время анализа
    # Сфорсмруем список признаков, которые исключаем из корреляционного анализа
    cols_to_drop = ['sample', 'city_copies',  'City_Amsterdam',  'City_Athens',  'City_Barcelona', 'City_Berlin',  'City_Bratislava',  'City_Brussels',  'City_Budapest',  'City_Copenhagen',  'City_Dublin',  'City_Edinburgh',  'City_Geneva',  'City_Hamburg',  'City_Helsinki', 'City_Krakow',  'City_Lisbon',  'City_Ljubljana',  'City_London',  'City_Luxembourg',  'City_Lyon',  'City_Madrid',  'City_Milan',  'City_Munich',  'City_Oporto',  'City_Oslo',  'City_Paris', 'City_Prague',  'City_Rome',  'City_Stockholm',  'City_Vienna',  'City_Warsaw',  'City_Zurich',  'City_nan', '',  'Afghani',  'African',  'Albanian',  'American',  'Arabic',  'Argentinean', 'Armenian',  'Asian',  'Australian',  'Austrian',  'Azerbaijani',  'Balti',  'Bangladeshi',  'Bar',  'Barbecue',  'Beer restaurants',  'Belgian',  'Brazilian',  'Brew Pub',  'British',  'Burmese',  'Cafe',  'Cajun & Creole',  'Cambodian',  'Campania',  'Canadian',  'Caribbean',  'Catalan',  'Caucasian',  'Central American',  'Central Asian',  'Central European',  'Central-Italian',  'Chilean',  'Chinese',  'Colombian',  'Contemporary',  'Croatian',  'Cuban',  'Czech',  'Danish',  'Deli',  'Delicatessen',  'Diner',  'Dining bars',  'Dutch',  'Eastern European',  'Ecuadorean',  'Egyptian',  'Emilian',  'Ethiopian',
                'European', 'Fast Food',  'Filipino',  'French',  'Fruit parlours',  'Fujian',  'Fusion',  'Gastropub',  'Georgian',  'German',  'Gluten Free Options',  'Greek',  'Grill',  'Halal',  'Hawaiian',  'Healthy',  'Hungarian',  'Indian',  'Indonesian',  'International',  'Irish',  'Israeli',  'Italian',  'Jamaican',  'Japanese',  'Japanese Fusion',  'Korean',  'Kosher',  'Latin',  'Latvian',  'Lazio',  'Lebanese',  'Lombard',  'Malaysian',  'Mediterranean',  'Mexican',  'Middle Eastern',  'Minority Chinese',  'Mongolian',  'Moroccan',  'Native American',  'Neapolitan',  'Nepali',  'New Zealand',  'Northern-Italian',  'Norwegian',  'Pakistani',  'Persian',  'Peruvian',  'Pizza',  'Polish',  'Polynesian',  'Portuguese',  'Pub',  'Romagna',  'Romana',  'Romanian',  'Russian',  'Salvadoran',  'Sardinian',  'Scandinavian',  'Scottish',  'Seafood',  'Sicilian',  'Singaporean',  'Slovenian',  'Soups',  'South American',  'Southern-Italian',  'Southwestern',  'Spanish',  'Sri Lankan',  'Steakhouse',  'Street Food',  'Sushi',  'Swedish',  'Swiss',  'Taiwanese',  'Thai',  'Tibetan',  'Tunisian',  'Turkish',  'Tuscan',  'Ukrainian',  'Uzbek',  'Vegan Options',  'Vegetarian Friendly',  'Venezuelan',  'Vietnamese',  'Welsh',  'Wine Bar',  'Xinjiang',  'Yunnan',  'Unknown']
    # Сформируем сет со скоррелированными признаками
    correlated_features = set()
    # Удаляем целевую переменную из матрицы коррелиций, тк корреляция с ней, - хорошо для модели
    correlation_matrix = df_output[df_output['sample'] == 1].drop(
        ['Rating', 'sample'], axis=1).corr()
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > 0.8:
                colname = correlation_matrix.columns[j]
                correlated_features.add(colname)

    # Сформируем сет для статистически незначимых признаков
    to_remove_features = set()
    # Проходим по колонкам, которые исключали из корреляционного анализа
    for column in cols_to_drop:
        to_remove_features.add(get_stat_dif_2(column))

    # Формируем сет, конвертируем в список, удаляем NAN
    drop_features = correlated_features.union(to_remove_features)
    drop_features = list(drop_features)
    drop_features.remove(None)

    df_output.drop(drop_features, axis=1, inplace=True)

    # Модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [
        s for s in df_output.columns if df_output[s].dtypes in ['object', '<M8[ns]']]
    df_output.drop(object_columns, axis=1, inplace=True)

    return df_output

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data, data_ta)
df_preproc.sample(10)

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

Перед тем, как отправлять  данные на обучение, разделим данные на еще один тест и трейн, для валидации. 

Это поможет проверить, как хорошо модель работает, до отправки submissiona на kaggle.

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# Проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

<a id="6"></a>

# 6. MODEL 

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Так как признак рейтинга имеет шаг 0.5, округляем предсказание.
y_pred = np.round(y_pred * 2) / 2

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
MAE = metrics.mean_absolute_error(y_test, y_pred)
print('MAE:', MAE)

Стартовое значение MAE: 0.428

Прогресс: 
* 0.214 (добавление осн. признаков)
* 0.2131 (добавление дамми инфы про кухни)
* 0.304 (если удалить Ranking и добавить ranking_quantile)
* 0.2186 (если оставить и Ranking, и ranling_quantile)
* 0.2186 (убирание выбросов по Number of Reviews никак не повлияло)
* 0.2186 (после нормализации ранга – без влияния)
* 0.213034375 (закомментировала ranling_quantile, улучшение)
* 0.21845 (public) - 0.213 - **440** место – пробный сабмит
* 0.20 - rank_per_ttl (v. 33) - **317** место - 0.21177
* 0.206870625 - добавила in_chain
* 0.207 - убрала выбросы в Num of Review и стало хуже
* v.34 - 0.207178125, 0.21202 (стало хуже) - **318** место - добавила выбросы по delta-reviews
* v.35 + ranking_num_reviews, reviews_per_ttl_ppl, ranking_num_cuisines (0.20595999999999998) **317** 0.21161
* v.36: подкорректировала границы выбросов, чтоб задеть меньше данных (0.20570312499999996)
* v.38: вернула признак is_Nan для ревью 0.20369375 (0.20880)
* v.39: добавила признак dietary_restrictions, локально ухудшила результат.
*  добавила date_rev_from_max 0.199690625 **0.20441** **233 место**
* v.40: reviews_perc_in_city_ttl 0.19911312499999997 **0.20427 233 место**
* v. 42: awards_num с TA: 0.196016875, **0.20106 223 место**
* v. 43: cousines from TA: 0.19679624999999998, **0.20123, 233 место** стало хуже!
* Округление шага в 0.5 0.1659375 0.17125 **0.17125, 65 место**
* Добавила координаты широты и долготы, поправила обработку кол-ва кухонь (на -1) 0.165875
* Заменила -1 на среднее по признаку 0.16625 **0.17255** хуже
* Откатила изменение **0.17270** Т.е. добавление широты и долготы не улучшает результат.
* v. 44 Отбор признаков **0.160125** **0.16695 29 место**
* Заполнение distance, 0.161875 хуже, но лучше в финалке **0.16675** 28 место
* 0.161 исправила баг с кухнями  0.16810
*  0.161937 - выбросы эксперименты
* v.45: 0.161  0.16810 - 28 место, финалка без «причесывания» кода
* v. 49 0.1595625 - Your submission scored 0.16810 - 28 место.


In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')

<a id="7"></a>

# 7. SUBMISSION 
Готовим Submission на Kaggle.

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
# Так как признак рейтинга имеет шаг 0.5, округляем предсказание.
predict_submission = np.round(predict_submission * 2) / 2

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

<a id="8"></a>

# 8. SUMMARY 

По ходу выполнения проекта:
1. Были добавлены внешние данные с информацией по городам и из TA
2. Были добавлены следующие признаки:
| Признак | Описание |
|-: |:- |
| number_of_rev_is_NAN | Наличие пропусков в изначальных данных по количеству отзывов | 
| awards_ta | Список наград ресторана с TA| 
| awards_num | Количество наград у ресторана с TA | 
| cuisine_styles_ta | Cписок с кухнями ресторана с TA| 
| longitude | Географические координаты ресторана с TA | 
| latitude | Географические координаты ресторана с TA|
| country  | Страна, в которой находится город |
| citizens | Население города, чел |
| restaurants_number_TA | Количество ресторанов, участвующих в рейтинге |
| citizens_per_restaurant | Количество горожан на один ресторан |
| tourists_per_year| Количество туристов, посетивших город в течение года, чел |
| ttl_ppl_per_restaurants | (Количество туристов + население города) / количество ресторанов |
| distance  | Расстояние от центра города до ресторана |
| reviews_per_ttl_ppl | Показывает, сколько ревью приходится на суммарных 1000 людей (жители + туристы) |
| ttl_reviews_per_city | Суммарное количество ревью по городу из выборки |
| reviews_perc_in_city_ttl | Отношения количества ревью ресторана к суммарному количеству ревью по городу из выборки |
| price_range_num | Ценовая категория ресторана: 1, 2, 3 |
| cuisine_num | Количество типов кухонь ресторана |
| dietary_restrictions | Наличие у ресторана спец. опций по кухням |
| review_date | Все даты ревью |
| date_rev_1 | Дата первого ревью |
| date_rev_2 | Дата второго ревью |
| in chain | Показатель, сетевой ли ресторан | 
| rank_per_ttl | показывает относительную позицию ранга ресторана к общему количеству рангов по городу |
| ranking_num_reviews | Умножение Ranking и Number of Reviews |
| ranking_num_cuisines | Умножение Ranking и cuisine_num |
| date_rev_delta | Количество дней между оставленными ревью |
| date_rev_from_max | Количество дней от последнего отзыва до самого свежего отзыва в датасете |
| City | Созданы 31 признак для кодировки города|
| Cuisine Style | Созданы 146 признака для кодировки типа кухни, включая отсуствие признака или пропуск |
3. После отбора признаков для модели были получены резултат:
    * Локальное MAE: 0.1595625
    * MAE на Kaggel (для submission): 0.16810 (28 место на момент отправки).